In [26]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import collections
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from wordcloud import WordCloud
from nltk.corpus import stopwords
%matplotlib inline

In [6]:
mails = pd.read_csv('spam.csv', encoding = 'latin-1')

In [7]:
mails.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN


In [8]:
mails.shape

(5572, 5)

In [9]:
mails.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'], axis = 1, inplace = True)

In [10]:
mails.head(1)

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."


In [11]:
mails.rename(columns = {'v1': 'spam', 'v2': 'message'}, inplace = True)
mails.head(1)

,spam,message
0,ham,"Go until jurong point, crazy.. Available only ..."


In [12]:
mails['spam'] = mails['spam'].map({'ham': False, 'spam': True})
mails.head()

,spam,message
0,False,"Go until jurong point, crazy.. Available only ..."
1,False,Ok lar... Joking wif u oni...
2,True,Free entry in 2 a wkly comp to win FA Cup fina...
3,False,U dun say so early hor... U c already then say...
4,False,"Nah I don't think he goes to usf, he lives aro..."


In [13]:
mails['spam'].value_counts()

False    4825
True      747
Name: spam, dtype: int64

In [14]:
all_mails_count = mails['spam'].count()
train_data = mails.loc[:all_mails_count*0.70]
train_data['spam'].value_counts()

False    3382
True      519
Name: spam, dtype: int64

In [15]:
display(train_data)

,spam,message
0,False,"Go until jurong point, crazy.. Available only ..."
1,False,Ok lar... Joking wif u oni...
2,True,Free entry in 2 a wkly comp to win FA Cup fina...
3,False,U dun say so early hor... U c already then say...
4,False,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
3896,False,No. Thank you. You've been wonderful
3897,False,Otherwise had part time job na-tuition..
3898,False,ÌÏ mean it's confirmed... I tot they juz say o...
3899,False,Okie


In [16]:
test_data = mails.loc[all_mails_count*0.70 + 1:]
test_data.reset_index(inplace = True)
test_data['spam'].value_counts()

False    1442
True      228
Name: spam, dtype: int64

In [17]:
display(test_data)

,index,spam,message
0,3902,False,Waiting in e car 4 my mum lor. U leh? Reach ho...
1,3903,True,Your 2004 account for 07XXXXXXXXX shows 786 un...
2,3904,True,Do you want a new video handset? 750 anytime a...
3,3905,False,Went fast asleep dear.take care.
4,3906,False,No that just means you have a fat head
...,...,...,...
1665,5567,True,This is the 2nd time we have tried 2 contact u...
1666,5568,False,Will Ì_ b going to esplanade fr home?
1667,5569,False,"Pity, * was in mood for that. So...any other s..."
1668,5570,False,The guy did some bitching but I acted like i'd...


In [19]:
def count_words(data):
    counter = collections.OrderedDict()
    for message in data:
        for word in message.split(' '):
            if word in counter:
                counter[word] += 1
            else:
                counter[word] = 1
    return counter

In [20]:
spam_messages = set(train_data[train_data['spam'] == True]['message'])
spam_words = count_words(spam_messages)
ham_messages = set(train_data[train_data['spam'] == False]['message'])
ham_words = count_words(ham_messages)

In [43]:
words = open('words.txt').read().split(' ')
wordlist = set(words)

In [44]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\shrey\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\shrey\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [45]:
def process_message(message):  
    words = message.lower() # lowercase
    words = word_tokenize(words) # tokenization
    words = [word for word in words if len(word) > 1] # non absurd words          
    words = [word for word in words if word not in stop_words] # non stop words
    words = [word for word in words if word in wordlist] # english words
    words = [PorterStemmer().stem(word) for word in words] # stemming
    return words

In [46]:
def count_processed_words(data):
    counter = collections.OrderedDict()
    for message in data:
        words = process_message(message)
        for word in set(words):           
            if word in counter:
                counter[word] += 1
            else:
                counter[word] = 1
    return counter

In [48]:
spam_words = count_processed_words(spam_messages)
ham_words = count_processed_words(ham_messages)

In [49]:
spam_messages = [i for i in spam_messages if len(process_message(i)) >= 1]
ham_messages = [i for i in ham_messages if len(process_message(i)) >= 1]

In [50]:
all_messages = spam_messages + ham_messages
all_words = count_processed_words(all_messages)

In [51]:
def spam(message, s=1, p=0.5, percentage=False):
    n = 0
    spam_freq = 0
    ham_freq = 0
    for word in process_message(message):
        if word in spam_words.keys(): 
            spam_freq = (spam_words[word] / all_words[word])
        if word in ham_words.keys():
            ham_freq = (ham_words[word] / all_words[word])
        if not (spam_freq + ham_freq) == 0 and word in all_words.keys(): 
            spaminess_of_word = (spam_freq ) / (spam_freq  + ham_freq )
            corr_spaminess = (s * p + all_words[word] * spaminess_of_word) / (s + all_words[word])            
            n += np.log(1 - corr_spaminess) - np.log(corr_spaminess)
    spam_result = 1 / (1 + np.e**n)
    if percentage:
        print(f'Spam probability: {spam_result*100:.2f}%')
    elif spam_result > 0.5:    
        return True
    else:
        return False

In [52]:
def test(spam_test, ham_test, s=1, p=0.5, details=False):
    spam_count = 0
    ham_count = 0
    for message in spam_test:
        if spam(message, s, p):
            spam_count += 1
        else:
            ham_count += 1

    true_positive = spam_count
    false_negative = ham_count

    spam_count = 0
    ham_count = 0
    for message in ham_test:
        if spam(message, s, p):
            spam_count += 1
        else:
            ham_count += 1
    
    false_positive = spam_count
    true_negative = ham_count

    # How many selected messages are spam?
    spam_precision = true_positive / (true_positive + false_positive)
    
    # How many spam messages are selected?
    spam_recall = true_positive / (true_positive + false_negative)
    
    # Harmonic mean between precision and recall.
    spam_fscore = 2 * (spam_precision * spam_recall) / (spam_precision + spam_recall) 

    # How many selected messages are ham?
    ham_precision = true_negative / (true_negative + false_negative)
    
    # How many ham messages are selected?
    ham_recall = true_negative / (true_negative + false_positive)
    
    # Harmonic mean between precision and recall.
    ham_fscore = 2 * (ham_precision * ham_recall) / (ham_precision + ham_recall)
    
    # If the data was ballanced.
    # accuracy = (true_positive + true_negative) / (true_positive + true_negative + false_positive + false_negative)
    
    # For unballanced data.
    ballanced_accuracy = ( spam_recall + ham_recall ) / 2

    if details:
        print("True Positive: ", true_positive)
        print("False Negative:", false_negative)
        print("True Negative: ", true_negative)
        print(f"False  Positive: {false_positive}\n")
        
        print(f"Spam precision: {spam_precision*100:.2f}%")
        print(f"Spam recall: {spam_recall*100:.2f}%")
        print(f"Spam F-score: {spam_fscore*100:.2f}%\n")
        
        print(f"Ham precision: {ham_precision*100:.2f}%")
        print(f"Ham recall: {ham_recall*100:.2f}%")
        print(f"Ham F-score: {ham_fscore*100:.2f}%\n")
        
    print(f"Accuracy: {ballanced_accuracy*100:.2f}%\n")